In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import preprocessing
import xgboost as xgb
import scipy.stats as stats
import seaborn as sns
import gc

from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

In [ ]:
train = pd.read_csv('grupo-bimbo-inventory-demand/train.csv')
test = pd.read_csv('grupo-bimbo-inventory-demand/test.csv')

print('Train and Test Read')

In [ ]:

train['target'] = train['Demanda_uni_equil']
train.drop(['Demanda_uni_equil'],axis=1, inplace = True)

train['tst'] = 0
test['tst'] = 1

data = pd.concat([train,test], axis=0, copy=True)

print('Train and Test Concat')
del train
del test
gc.collect()

In [ ]:
for i in range(1,6):
    lag = 'Lag' + str(i)
    print('Lag:',lag)

    data1 = data[['Semana','Cliente_ID','Producto_ID','target']]
    data1.loc[:,'Semana'] = data1['Semana'] +i
    data1 = data1.groupby(['Semana','Cliente_ID','Producto_ID']).mean()
    data1 = data1.reset_index()
    data1.rename(columns={'target': lag}, inplace=True)
    data = pd.merge(data,data1,
                    how='left',
                    left_on=['Semana','Cliente_ID','Producto_ID'],
                    right_on=['Semana','Cliente_ID','Producto_ID'],
                    left_index=False, right_index=False, sort=True,
                    suffixes=('_x', '_y'), copy=False, )
    del data1
    gc.collect()

In [ ]:

data['TotalLags'] = data['Lag1'] + data['Lag2']+ data['Lag3']+ data['Lag4']+ data['Lag5']

data=data[data['Semana']>8]  # NOW I WORK WITH WEEKS 9,10,11

nAgencia = pd.DataFrame(data.groupby(['Agencia_ID','Semana'])['target'].count())
nAgencia = nAgencia.reset_index()
nAgencia.rename(columns={'target': 'nAgencia'}, inplace=True)
nAgencia = pd.DataFrame(nAgencia.groupby(['Agencia_ID'])['nAgencia'].mean())
nAgencia = nAgencia.reset_index()


data = pd.merge(data, nAgencia,
                how='left',
                left_on=['Agencia_ID'],
                right_on=['Agencia_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nAgencia
gc.collect()
print('merge completo nAgencia')
print(data.shape[0])

nRuta_SAK = pd.DataFrame(data.groupby(['Ruta_SAK','Semana'])['target'].count())
nRuta_SAK = nRuta_SAK.reset_index()
nRuta_SAK.rename(columns={'target': 'nRuta_SAK'}, inplace=True)
nRuta_SAK = pd.DataFrame(nRuta_SAK.groupby(['Ruta_SAK'])['nRuta_SAK'].mean())
nRuta_SAK = nRuta_SAK.reset_index()


data = pd.merge(data, nRuta_SAK,
                how='left',
                left_on=['Ruta_SAK'],
                right_on=['Ruta_SAK'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nRuta_SAK
gc.collect()
print('merge completo nRuta_SAK')
print(data.shape[0])

nCliente_ID = pd.DataFrame(data.groupby(['Cliente_ID','Semana'])['target'].count())
nCliente_ID = nCliente_ID.reset_index()
nCliente_ID.rename(columns={'target': 'nCliente_ID'}, inplace=True)
nCliente_ID = pd.DataFrame(nCliente_ID.groupby(['Cliente_ID'])['nCliente_ID'].mean())
nCliente_ID = nCliente_ID.reset_index()


data = pd.merge(data, nCliente_ID,
                how='left',
                left_on=['Cliente_ID'],
                right_on=['Cliente_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nCliente_ID
gc.collect()
print('merge completo nCliente_ID')
print(data.shape[0])

nProducto_ID = pd.DataFrame(data.groupby(['Producto_ID','Semana'])['target'].count())
nProducto_ID = nProducto_ID.reset_index()
nProducto_ID.rename(columns={'target': 'nProducto_ID'}, inplace=True)
nProducto_ID = pd.DataFrame(nProducto_ID.groupby(['Producto_ID'])['nProducto_ID'].mean())
nProducto_ID = nProducto_ID.reset_index()


data = pd.merge(data, nProducto_ID,
                how='left',
                left_on=['Producto_ID'],
                right_on=['Producto_ID'],
                left_index=False, right_index=False, sort=True,
                suffixes=('_x', '_y'), copy=False)

del nProducto_ID
gc.collect()
print('merge completo nProducto_ID')
print(data.shape[0])

data.replace(np.nan,0, inplace=True)

train = data[data['tst']==0]
predict = data[data['tst']==1]

train['target'] = np.log(train['target'] + 1)
#train2 = train.sample(n=1000000)   <-- another possible reduction of data for fast testing
train2 = train
y = train['target']
X = train[[  'Agencia_ID','Canal_ID','Cliente_ID','Producto_ID','Ruta_SAK',
             'Lag1','Lag2','Lag3','Lag4','Lag5','TotalLags',
             'nAgencia','nRuta_SAK','nCliente_ID','nProducto_ID']]

print(X.shape, y.shape)

In [ ]:
# X.to_csv('x.csv',index=False)
# y.to_csv('y.csv',index=False)
train.to_csv('data.csv',index=False)

In [ ]:
# sns.palplot(train)

In [ ]:
X=pd.read_csv('x.csv')
y=pd.read_csv('y.csv')
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=1729)
print(X_train.shape, X_test.shape)

In [ ]:
xlf = xgb.XGBRegressor(max_depth=10,
                       learning_rate=0.1,
                       n_estimators=10,
                       silent=True,
                       objective="reg:squarederror",
                       nthread=-1,
                       gamma=0,
                       min_child_weight=1,
                       max_delta_step=0,
                       subsample=0.85,
                       colsample_bytree=0.7,
                       colsample_bylevel=1,
                       reg_alpha=0,
                       reg_lambda=1,
                       scale_pos_weight=1,
                       seed=1440,
                       missing=1,
                       eval_metric='rmsle')

In [ ]:
xlf.fit(X_train, y_train, verbose = True, eval_set = [(X_test, y_test)])

In [ ]:
# calculate the auc score
preds = xlf.predict(X_test)

print('\nRoot Mean Square error of Log(Demanda) X_test" ', mean_squared_error(y_test,preds)**0.5)


In [ ]:
lgbm_regr = lgb.LGBMRegressor(num_leaves=1000,
                              max_depth=10,
                              random_state=42)

lgbm_regr.fit(X_train,y_train,verbose=True,eval_set=(X_test,y_test))

In [ ]:
preds2 = lgbm_regr.predict(X_test)

print('\nRoot Mean Square error of Log(Demanda) X_test" ', mean_squared_error(y_test,preds2)**0.5)